In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import xtrack as xt
import matplotlib.pyplot as plt

In [ ]:
line = xt.Line.from_json('pimms_00_optics.json')
line.configure_bend_model(edge='full', core='adaptive', num_multipole_kicks=10)

In [ ]:

line.build_tracker()

In [ ]:
line.vars['kse1'] = 1
line.vars['kse2'] = -6.5

In [ ]:
from phase_space_characterization import characterize_phase_space_at_septum

In [ ]:
%time characterize_phase_space_at_septum(line, num_turns=1000)

In [ ]:
characterize_phase_space_at_septum(line, num_turns=1000, plot=True)

In [ ]:
class ActionSeparatrix(xt.Action):
    
    def __init__(self, line):
        self.line = line
        
    def run(self):
        out = characterize_phase_space_at_septum(self.line)
        return out

In [ ]:
action = ActionSeparatrix(line)

In [ ]:
action.run()

In [ ]:
opt = line.match(
    solve=False,
    method='4d',
    vary=xt.VaryList(['kse1', 'kse2'], step=0.5, limits=[-7, 7]),
    targets=[
        action.target('stable_area', 1.e-4, tol=1e-5, weight=100), 
        action.target('dpx_dx_at_septum', 0.03, tol=5e-4)
    ]
)

In [ ]:
opt.target_status()

In [ ]:
def err_fun(x):
    out = opt._err(x, check_limits=False)
    #print(f'x = {repr(x)}, out = {out}')
    return out

bounds = np.array([vv.limits for vv in opt._err.vary])
opt._err.return_scalar = True
import pybobyqa
soln = pybobyqa.solve(err_fun, x0=opt.log().vary[0, :], bounds=bounds.T,
            rhobeg=5, rhoend=1e-4, maxfun=30, objfun_has_noise=True,
            seek_global_minimum=True)
err_fun(soln.x) # set it to found solution
opt.tag('pybobyqa') # Add point to optimization log
opt.target_status()

In [ ]:
opt.log()

In [ ]:
opt.vary_status()

In [ ]:
characterize_phase_space_at_septum(line, num_turns=1000, plot=True)

In [ ]:
line.to_json('pimms_02_tuned.json')